# Import Libraries

In [3]:
import random
import pandas as pd
import numpy as np
import contractions
import re
from nltk.tokenize import word_tokenize
from nltk.stem.snowball import SnowballStemmer
from pymorphy2 import MorphAnalyzer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.snowball import SnowballStemmer

import fasttext.util

import time

import torch

from transformers import AutoTokenizer, AutoModel

from lightgbm import LGBMClassifier

from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

In [4]:
# Загружаем стоп-слова
RU_STOPWORDS = stopwords.words("russian")
EN_STOPWORDS = stopwords.words("english")

In [5]:
# Фиксируем seed для воспроизводимости
def seed_all(seed: int) -> None:
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

seed_all(42)

# Load Data

In [6]:
df = pd.read_csv('starter/artefacts/data/df.csv')
df.head()

,Text,Label
0,Арбитражный суд Ханты-Мансийского автономного ...,Финансы компаний
1,Группа «Онэксим» Михаила Прохорова переуступил...,Финансы компаний
2,"Миноритарный пакет «Башнефти», остающийся у АФ...",Финансы компаний
3,Крупнейшие российские госкомпании-экспортеры —...,Финансы компаний
4,"Инвестиционная компания «Авиализинг», подавшая...",Финансы компаний


In [7]:
# Инициализация LabelEncoder
label_encoder = LabelEncoder()

# Применение к столбцу таргетов
df['Label'] = label_encoder.fit_transform(df['Label'])
df.tail()

,Text,Label
1789,Нужно изолировать каклов с либерахами и желате...,1
1790,ФАЯЯЯЯЯЯ ТУТТУТУТУТТУТУТУТРУРУТУРУТУ\n,1
1791,"Я ваш коммент курсором отпиздил, настолько он ...",1
1792,ПЕДРОНЫ НА МЕСТЕ? СОТОНИСТЫ ОЧКАРИКИ ЗДЕСЬ?\n,1
1793,"Не просто портить жизнь , тот малолетний уебан...",1


In [8]:
# Сэмплирование строк для тестовой выборки
train_df = df.groupby('Label').apply(lambda x: x.sample(n=15, replace=True))

# Сохранение индексов 
sampled_indices = [i[1] for i in list(train_df.index)]

# Сброс индекса
train_df = train_df.reset_index(drop=True)

# Удаление сэмплированных строк из тренировочной выборки
test_df = df.drop(sampled_indices).reset_index(drop=True)

# Preprocessing

In [9]:
def preprocess_text(text: str) -> str:
    """
    Функция для предобработки текста.
    """
    if not isinstance(text, str):
        return ''

    # Приводим текст к нижнему регистру
    text = text.lower()

    # Разворачиваем сокращения: текст часто содержит конструкции на англ. языке, вроде "don't" или "won't"
    text = contractions.fix(text)

    # Разделяем "слипшиеся" с пунктуацией слова
    text = re.sub(r'\s*[.,;:!?…-]+\s*', ' ', text)
    
    # Удаляем пунктуацию, не разделяя слова и знаки препинания
    text = re.sub(r'[^a-zA-Zа-яА-ЯёЁ0-9\s]', '', text)

    # Удаляем лишние пробелы
    text = re.sub(r'\s+', ' ', text).strip()

    # Удаляем стоп-слова
    text = ' '.join([word for word in text.split() if word not in RU_STOPWORDS or word not in EN_STOPWORDS])
    
    # Заменяем дефисы на пробелы 
    text = re.sub(r'(\w+)-(\w+)', r'\1 \2', text)

    # Создаем экземпляр морфологического анализатора
    morph = MorphAnalyzer()

    # Создание стеммера
    stemmer = SnowballStemmer("english", ignore_stopwords=True)

    # Токенизируем текст для более точного разделения на слова
    words = word_tokenize(text, language='russian')

    # Инициализация списка для нормализованных слов
    processed_words = []

    for word in words:
        # Определяем язык слова
        if bool(re.fullmatch(r'[a-zA-Z]+', word)):
            # Выделим основу слова
            parsed_word = stemmer.stem(word)
            processed_words.append(parsed_word)             
        else:
            # Применяем лемматизацию
            parsed_word = morph.parse(word)[0]
            processed_words.append(parsed_word.normal_form)
        
    # Соединяем обработанные слова в строку
    return ' '.join(processed_words)

In [10]:
# Препроцессинг тренировочной выборки
train_df['Preproced_text'] = train_df['Text'].apply(preprocess_text)
train_df.dropna(inplace=True)
train_df.sample(10)

,Text,Label,Preproced_text
15,Пыня тоже геноцидил чеченцев во вторую кампани...,1,пынь тоже геноцидить чеченец в второй кампания...
9,"А в 1960-м рубль был дороже доллара, а интерне...",0,а в 1960 м рубль быть дорогой доллар а интерне...
99,Основной акционер банка «Уралсиб» Николай Цвет...,6,основной акционер банк уралсиб николай цветков...
80,Популярное Android-приложение «WiFi Finder» ра...,5,популярный android приложение wifi finder раск...
68,Руководство «Манчестер Юнайтед» планирует уста...,4,руководство манчестер юнайтед планировать уста...
94,«Аэрофлот» обратился в Арбитражный суд Москвы ...,6,аэрофлот обратиться в арбитражный суд москва с...
59,За пять месяцев 2016 года на отечественном авт...,3,за пять месяц 2016 год на отечественный авторы...
17,"Заебали, вам смешно, а мне из за этого долбоеб...",1,заебали вы смешно а я из за это долбоеб прийти...
49,Ульяновский автомобильный завод начинает отзыв...,3,ульяновский автомобильный завод начинать отзыв...
37,"Юхана Фронен, заведующий нейрохирургическим от...",2,юхан фронный заведовать нейрохирургический отд...


In [11]:
# Препроцессинг тестовой выборки
test_df['Preproced_text'] = test_df['Text'].apply(preprocess_text)
test_df.dropna(inplace=True)
test_df.sample(10)

,Text,Label,Preproced_text
857,Специалист по кибербезопасности Джонатан Лейтш...,5,специалист по кибербезопасность джонатан лейтш...
1480,"Хм, знакомый врач, лет пять назад закончил мед...",0,хм знакомый врач год пять назад закончить мёд ...
56,Показатель EBITDA группы компаний «Аэрофлот» в...,6,показатель ebitda группа компания аэрофлот в п...
1100,Международная федерация футбола (ФИФА) обязала...,4,международный федерация футбол фифа обязать ва...
859,У шпионской программы FinSpy появились новые в...,5,у шпионский программа finspi появиться новый в...
870,Специалисты в области кибербезопасности из ком...,5,специалист в область кибербезопасность из комп...
705,"Американская компания FireEye, специализирующа...",5,американский компания fireey специализироватьс...
584,Жителя Великобритании приговорили к 28 неделям...,5,житель великобритания приговорить к 28 неделя ...
540,Американские хакеры-студенты Далтон Норман (Da...,5,американский хакер студент далтон норман dalto...
979,«Лестер» на своем поле обыграл «Борнмут» в мат...,4,лестер на свой поле обыграть борнмут в матч че...


In [12]:
# Разбиение на трейн и тест
X_train, y_train = train_df['Preproced_text'], train_df['Label']
X_test, y_test = test_df['Preproced_text'], test_df['Label']

# FastText + Classic models

In [13]:
fasttext.util.download_model('ru', if_exists='ignore') 
ft = fasttext.load_model('starter/artefacts/data/cc.ru.300.bin')

In [14]:
# Определение моделей
models = {
    "Naive Bayes": make_pipeline(MinMaxScaler(), MultinomialNB()),
    "SVM": make_pipeline(StandardScaler(), SVC(kernel='linear', probability=True)),  
    "Logistic Regression": make_pipeline(StandardScaler(), LogisticRegression()),
    "KNeighborsClassifier": make_pipeline(StandardScaler(), KNeighborsClassifier()),
    "Random Forest": RandomForestClassifier(),
    "LGBMClassifier": LGBMClassifier(class_weight='balanced', n_jobs=-1, verbose=-1)
}

In [15]:
# Оценка моделей с предобученными векторами FastText по всем метрикам
for model_name, model in models.items():
    start_time = time.time()
    X_train_ft = np.array([ft.get_sentence_vector(text) for text in X_train]).squeeze()
    X_test_ft = np.array([ft.get_sentence_vector(text) for text in X_test]).squeeze()
    model.fit(X_train_ft, y_train)
    y_pred = model.predict(X_test_ft)
    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"Classification Report for {model_name}(Time taken: {elapsed_time:.4f} seconds):")
    print(classification_report(y_test, y_pred))

Classification Report for Naive Bayes(Time taken: 0.7859 seconds):
              precision    recall  f1-score   support

           0       0.81      0.68      0.74       186
           1       0.86      0.69      0.77       185
           2       0.87      0.97      0.92       191
           3       0.90      0.81      0.86       295
           4       0.96      0.97      0.96       333
           5       0.96      0.95      0.95       362
           6       0.62      0.94      0.75       142

    accuracy                           0.87      1694
   macro avg       0.85      0.86      0.85      1694
weighted avg       0.88      0.87      0.87      1694

Classification Report for SVM(Time taken: 0.8423 seconds):
              precision    recall  f1-score   support

           0       0.80      0.59      0.68       186
           1       0.73      0.78      0.76       185
           2       0.90      0.97      0.93       191
           3       0.90      0.93      0.92       295
      

# Bert embeddings + Classic models

In [16]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [17]:
# Загружаем модель и токенайзер с https://huggingface.co/ai-forever/sbert_large_mt_nlu_ru
tokenizer = AutoTokenizer.from_pretrained("ai-forever/sbert_large_mt_nlu_ru")
model = AutoModel.from_pretrained("ai-forever/sbert_large_mt_nlu_ru")

tokenizer_config.json:   0%|          | 0.00/1.24k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.78M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.71M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/866 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.71G [00:00<?, ?B/s]

In [18]:
model.to(device)

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(120138, 1024, padding_idx=0)
    (position_embeddings): Embedding(512, 1024)
    (token_type_embeddings): Embedding(2, 1024)
    (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-23): 24 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=1024, out_features=1024, bias=True)
            (key): Linear(in_features=1024, out_features=1024, bias=True)
            (value): Linear(in_features=1024, out_features=1024, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=1024, out_features=1024, bias=True)
            (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1,

In [19]:
# Функция для получения эмбеддингов 
def get_bert_embeddings(text):
    inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True).to(device) 
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.pooler_output.cpu().numpy() 

In [20]:
# Оценка эмбеддингов Berta c моделями по всем метрикам
for model_name, model in models.items():
    start_time = time.time()
    X_train_bert = np.array([ft.get_sentence_vector(text) for text in X_train]).squeeze()
    X_test_bert = np.array([ft.get_sentence_vector(text) for text in X_test]).squeeze()
    model.fit(X_train_bert, y_train)
    y_pred_bert = model.predict(X_test_bert)
    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"Classification Report for {model_name}(Time taken: {elapsed_time:.4f} seconds):")
    print(classification_report(y_test, y_pred))

Classification Report for Naive Bayes(Time taken: 0.8442 seconds):
              precision    recall  f1-score   support

           0       0.77      0.42      0.54       186
           1       0.66      0.82      0.73       185
           2       0.92      0.95      0.94       191
           3       0.86      0.93      0.89       295
           4       0.96      0.97      0.97       333
           5       0.95      0.95      0.95       362
           6       0.77      0.81      0.79       142

    accuracy                           0.87      1694
   macro avg       0.84      0.84      0.83      1694
weighted avg       0.87      0.87      0.86      1694

Classification Report for SVM(Time taken: 0.8832 seconds):
              precision    recall  f1-score   support

           0       0.77      0.42      0.54       186
           1       0.66      0.82      0.73       185
           2       0.92      0.95      0.94       191
           3       0.86      0.93      0.89       295
      